In [1]:
import pandas as pd
import random
import csv

match_data = pd.read_csv("/home/mehul/Downloads/New_BData/matches/match2.csv")

In [2]:
match_data.head()

,T1 batsman,T2 bowler,T2 batsman,T1 bowler
0,C Munro,R Ashwin,KL Rahul,TA Boult
1,G Gambhir,MM Sharma,MA Agarwal,Mohammed Shami
2,SS Iyer,Mujeeb Ur Rahman,Yuvraj Singh,A Mishra
3,V Shankar,AR Patel,KK Nair,CH Morris
4,RR Pant,AJ Tye,DA Miller,DT Christian


In [3]:
T1_batsman = match_data['T1 batsman']
T1_bowler = match_data['T1 bowler']
T2_batsman = match_data['T2 batsman']
T2_bowler = match_data['T2 bowler']


In [4]:
score_list = [0, 1, 2, 3, 4, 6]

In [5]:
def get_cluster(batsman,bowler):
    bat_cluster_data = pd.read_csv("/home/mehul/Downloads/New_BData/predict/batsman_cluster_final.csv",header = 0).drop_duplicates()
    bowl_cluster_data = pd.read_csv("/home/mehul/Downloads/New_BData/predict/bowler_cluster_final.csv",header = 0).drop_duplicates()
    
    bat_cluster = bat_cluster_data.loc[bat_cluster_data['batsman']==batsman]
    bowl_cluster = bowl_cluster_data.loc[bowl_cluster_data['bowler']==bowler]
    #print(bowl_cluster)
    return bat_cluster['cluster'].item(),bowl_cluster['cluster'].item()

In [6]:
get_cluster(T1_batsman[5],T2_bowler[1])

(9, 0)

In [7]:
def pvp_profile(batsman,bowler):
    pvpData = pd.read_csv("/home/mehul/Downloads/New_BData/predict/pvpProb.csv",header = 0)
    data = pvpData[(pvpData.batsman == batsman)&(pvpData.bowler == bowler)]
    flag = True
    if data.empty:
        flag = False
        #print("dataFrame empty")
        return flag,None
    else:
        a = data.iloc[:,2:10].values.tolist()
        return flag,a[0]

In [8]:
#check,pvp_data = pvp_profile(T1_batsman[1],T2_bowler[0])

In [9]:
#print(pvp_data)

In [10]:
def cvc_profile(bat_cluster,bowl_cluster):
    cvcData = pd.read_csv("/home/mehul/Downloads/New_BData/predict/CvCprob.csv",header = 0)
    data = cvcData[(cvcData.bat_cluster== bat_cluster)&(cvcData.bowl_cluster == bowl_cluster)]
    if data.empty:
        #print("dataFrame empty")
        return None
    else:
        a = data.iloc[:,2:10].values.tolist()
        return a[0]

In [11]:
#cvc_data = cvc_profile(bat_cluster,bowl_cluster)

In [12]:
#print(cvc_data)

In [13]:
def random_pick(some_list, probabilities) :

    #print(len(probabilities))
    del(probabilities[5])
    del(probabilities[-1])
    x = random.uniform(0,sum(probabilities))
    cumulative_probability = 0.0
    for item, item_probability in zip(some_list, probabilities):
        cumulative_probability += item_probability
        if x < cumulative_probability: break
    return item

In [14]:

def match(bat_order, bow_order, inn) :
    tot_wickets = 0
    striker = 1
    non_striker = 0
    striker_notout = 1
    non_striker_notout = 1
    

    bow_index_order = [0,1,0,1,2,4,3,2,3,4,2,3,4,1,3,4,0,2,0,1]#bowling order
    x = bow_index_order[0]

    total_runs = 0
    k = -1
    print(bat_order[non_striker].rstrip())
    print(bat_order[striker].rstrip())
    for i in range(0,120) :
        # Swap batsmen and Change bowlers after every over
        score_sheet = [0]*5
        if i%6 == 0 :
            k += 1
            x = bow_index_order[k]

            tmp_striker = striker
            tmp_notout = striker_notout
            striker = non_striker
            striker_notout = non_striker_notout
            non_striker = tmp_striker
            non_striker_notout = tmp_notout

        curr_bat = bat_order[striker].rstrip() #Striker
        #print(curr_bat)
        score_sheet[0] = curr_bat
        other_bat = bat_order[non_striker].rstrip()# Non Striker
        #print(other_bat)
        score_sheet[1] = other_bat
        curr_bow = bow_order[x].rstrip()
        score_sheet[2] = curr_bow

        #Prediction
        flag,pvp_p_list = pvp_profile(curr_bat, curr_bow)
        if (flag == True) :
            striker_notout *= float(1 - (pvp_p_list[7]))
            prediction = random_pick(score_list, pvp_p_list)
            #print(pvp_p_list[7])
            #m_notout *= float(1 - (pvp_p_list[7]))
        else :
            #print(curr_bow)
            bat_c_num, bow_c_num = get_cluster(curr_bat, curr_bow)
            cvc_p_list = cvc_profile(bat_c_num, bow_c_num)
            striker_notout *= float(1 - (cvc_p_list[7]))
            prediction = random_pick(score_list, cvc_p_list)
            score_sheet[3] = prediction
            #print(cvc_p_list[7])
            #m_notout *= float(1 - (cvc_p_list[7]))

        #If out
        if striker_notout<0.4 :
            tot_wickets+=1
            striker=max(striker,non_striker) + 1
            score_sheet[4] = 1
            print(bat_order[striker].rstrip())
            striker_notout = 1

        if striker > 10:
            break

        elif prediction==0 or prediction==2 or prediction==4 or prediction==6:
            total_runs+=prediction


        elif prediction==1 or prediction==3:
            total_runs+=prediction
            tmp_striker = striker
            tmp_notout = striker_notout
            striker = non_striker
            striker_notout = non_striker_notout
            non_striker = tmp_striker
            non_striker_notout = tmp_notout



        if inn == 1 :
            global first_inn_score
            first_inn_score = total_runs

        # If it is second innings and if the team has chased down the target
        if inn == 2 and total_runs > first_inn_score :
            break
        #print(score_sheet) #UNCOMMENT TO PRINT BALL BY BALL STATS

    num_of_overs_played = str(int((i+1)/6)) + "." + str((i+1)%6)
    return total_runs, tot_wickets, str(total_runs)+"/"+str(tot_wickets)+" Overs : "+ num_of_overs_played


In [15]:
first_innings_score, wickets1, formatted_score1 = match(T1_batsman, T2_bowler, 1)
print ("Team 1 Score : " + formatted_score1)
print("\n*****************************\n")

second_innings_score, wickets2, formatted_score2 = match(T2_batsman, T1_bowler, 2)
print ("Team 2 Score : " + formatted_score2)

if first_innings_score > second_innings_score :
    print ("Team 1 wins!")
    print ("By "+str(first_innings_score - second_innings_score)+" runs")
elif second_innings_score > first_innings_score :
    print ("Team 2 wins!")
    print ("By "+ str(10 - wickets2) +" wickets")
else :
    print ("Match Tied.")

C Munro
G Gambhir
SS Iyer
V Shankar
RR Pant
R Tewatia
Team 1 Score : 133/4 Overs : 20.0

*****************************

KL Rahul
MA Agarwal
Yuvraj Singh
KK Nair
Team 2 Score : 134/2 Overs : 13.4
Team 2 wins!
By 8 wickets
